In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing

In [37]:
np.set_printoptions(precision=2, linewidth=80, suppress=True)

In [38]:
def normalize(data):
    # Fit() computes the min and max of each feature
    min_max_scaler = preprocessing.MinMaxScaler().fit(data)
    
    # Scaling features of X according to feature_range.
    spotify_df_normalized = min_max_scaler.transform(data)
    
    return spotify_df_normalized


In [43]:
# show me the name of the song
def show_me_the_song_name(data, song_id):
    return data.loc[data.index == song_id, ['name', 'artist', 'uri']]

In [40]:
def knn_search(song_id, data, K):
    """ find K nearest neighbors of a song among the instances in the dataset """

    song = data[song_id]
    data = np.delete(arr=data, obj=song_id, axis=0)
    
    data_norm = np.array([np.linalg.norm(data[i]) for i in range(len(data))])
    song_norm = np.linalg.norm(song)
    sims = np.dot(data,song)/(data_norm * song_norm)
    dists = 1 - sims    
    idx = np.argsort(dists) # sorting

    return idx[:K]

In [41]:
def get_feature_weights(normalized_Data, nn_indices):
    return normalized_Data[nn_indices.tolist(), :].mean(axis=0)

In [42]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import urllib, cStringIO

def create_wordcloud(features, feature_weights):
    
    words = {}
    for i in range(len(features)):
        words[features[i]] = feature_weights[i]

    file = cStringIO.StringIO(urllib.urlopen("http://bmobasher.com/Class/cloud2.jpg").read())
    img = Image.open(file)
    
    alice_coloring = np.array(img)
    
    wc = WordCloud(background_color="white", max_words=200, mask=alice_coloring, max_font_size=65, random_state=42)
    # generate word cloud
    wc.generate_from_frequencies(words)

    # create coloring from image
    image_colors = ImageColorGenerator(alice_coloring)

    fig = plt.figure(1)
    fig.set_figwidth(15)
    fig.set_figheight(9)
    plt.imshow(wc, interpolation="bilinear")

    plt.axis('off')
    plt.show()
    return fig